In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp

def find_if_close(cnt1,cnt2,const):
    M1 = cv2.moments(cnt1)
    M2 = cv2.moments(cnt2)
    
    if(M1["m00"]!=0 and M2["m00"]!=0):
        cX1 = int((M1["m10"] / M1["m00"]))
        cY1 = int((M1["m01"] / M1["m00"]))
        cX2 = int((M2["m10"] / M2["m00"]))
        cY2 = int((M2["m01"] / M2["m00"]))
        
        dist = np.sqrt((cX1-cX2)*(cX1-cX2)+(cY1-cY2)*(cY1-cY2)) 
        if dist < const :
            return True,(cX1,cY1),(cX2,cY2)
    return False,(0,0),(0,0)        
def saveMatInDir(mat,dir,name):
    cv2.imwrite(dir+name, mat)

    
def preproc_mat(img):
    if(len(img.shape)==2):
        img=np.array([[[0.] if not i else [1.] for i in j] for j in img])
    temp=img.transpose((2, 0, 1))
    return temp[None,:80,:80,...]

def saveMatToDataset(mat,classnum,dataType):
    index=0
    fileName="colored_dataset"
    if(dataType=="gray"):
        index=1
        fileName="gray_dataset"
    elif(dataType=="bgmask"):
        index=2
        fileName="bgmask_dataset_terkom"
    x_values[index].append(preproc_mat(mat))
    y_values[index].append(np.array([1. if classnum==i else 0. for i in range(classCount)]))
    mem=(x_values[index],y_values[index])
    np.save(fileName,mem)
    
def saveMatInDatasetFile(mat,classnum):
    if(mat.shape[:2]!=(80,80)):
        print("Error!: shape is not good"+str(mat.shape))
        return
    if(len(mat.shape)!=2):
        saveMatToDataset(mat,classnum,"colored")
        gray_mat = cv2.cvtColor(mat, cv2.COLOR_BGR2GRAY)
        saveMatToDataset(gray_mat,classnum,"gray")
    else:
        saveMatToDataset(mat,classnum,"bgmask")

def getClassName(classnum):
    if(classnum==0):
        return "ball"
    elif(classnum==1):
        return "player"
    elif(classnum==2):
        return "player+ball"
    elif(classnum==3):
        return "trash"

def dataset_tune():
        dataTypeInd=1
        ind=0
        for index,it in enumerate(x_values):
            if(len(it)>0):
                dataTypeInd=index
        if(dataTypeInd<0):
            return
        while(1):
            if(len(x_values[dataTypeInd])>0):
                blank_image = np.zeros((100,100,x_values[dataTypeInd][ind][0].shape[0]),
                                       x_values[dataTypeInd][ind][0].dtype)
                curmat=x_values[dataTypeInd][ind][0].transpose(1,2,0)
                #print(blank_image.shape)
                blank_image[20:,20:]=curmat
                cv2.putText(blank_image,"class="+getClassName(np.where(y_values[dataTypeInd][ind]==(1.))[0][0]),
                            (0,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            else:
                blank_image=np.zeros((100,100,3),dtype=np.uint8)
                cv2.putText(blank_image,"no data",(0,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.imshow('tuning',blank_image)
            key = cv2.waitKey(0)
            if(key==27):
                cv2.destroyWindow('tuning')
                return
            elif(key==2555904):
                if(ind==len(x_values[dataTypeInd])-1):
                    ind=0
                else:
                    ind+=1
            elif(key==2424832):
                if(ind==0):
                    ind=len(x_values[dataTypeInd])-1
                else:
                    ind-=1
            elif(key==2490368):
                if(dataTypeInd==len(x_values)-1):
                    dataTypeInd=0
                else:
                    dataTypeInd+=1
            elif(key==2621440):
                if(dataTypeInd==0):
                    dataTypeInd=len(x_values)-1
                else:
                    dataTypeInd+=1
            elif(key==ord('q')):
                y_values[dataTypeInd][ind]=np.array([1. if 0==i else 0. for i in range(classCount)])
            elif(key==ord('w')):
                y_values[dataTypeInd][ind]=np.array([1. if 1==i else 0. for i in range(classCount)])
            elif(key==ord('e')):
                y_values[dataTypeInd][ind]=np.array([1. if 2==i else 0. for i in range(classCount)])
            elif(key==ord('r')):
                y_values[dataTypeInd][ind]=np.array([1. if 3==i else 0. for i in range(classCount)])
        cv2.destroyWindow('tuning')
    
x_values=([],[],[])
y_values=([],[],[])

classCount=2

#cap = cv2.VideoCapture('C:\\Users\\egor\\Documents\\20161208_074126.mp4')
#cap = cv2.VideoCapture('20160519_081947.mp4')

#cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
#cap.set(cv2.CAP_PROP_POS_MSEC,161000)
#cap.set(cv2.CAP_PROP_POS_MSEC,1510000)
fgbg = cv2.createBackgroundSubtractorMOG2(100,5,False)
perFrame=True
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
iii = 30
frame = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\R_try2\\'+str(iii).zfill(9)+".png",0)
fgmask = fgbg.apply(frame)
iii+=1
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
oldUnified=[]
unified = []
linearr=[]
counter=0
while(1):
    iii+=1
    #ret, frame = cap.read()
    frame = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\R_try2\\'+str(iii).zfill(9)+".png",0)
    baseframe=frame.copy()
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    black = np.zeros_like(fgmask)
    res=np.zeros_like(frame)
    #res = cv2.bitwise_and(frame,frame,mask = fgmask)
   
    #fgmask = cv2.dilate(fgmask, kernel1, iterations=5)

    ret,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)>100):
        continue
   
    
    #######
    LENGTH = len(contours)
    status = np.zeros((LENGTH,1))

    for i,cnt1 in enumerate(contours):
        x = i    
        if i != LENGTH-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist,bbbb,bbbbbb = find_if_close(cnt1,cnt2,50)
                if dist == True:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1
    sizeold=len(linearr)
    rectArr=[]
    
    if(len(status>0)):
        oldUnified=list(unified)
        unified=[]
        maximum = int(status.max())+1
        for i in range(maximum):
            pos = np.where(status==i)[0]
            if pos.size != 0:
                cont = np.vstack(contours[i] for i in pos)
                x,y,w,h = cv2.boundingRect(cont)
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
                rectArr.append((x,y,w,h))
    if(len(rectArr)>50):
        continue
    cv2.imshow('asdasdf',fgmask)
    if(perFrame==True):
        for ind in range(len(rectArr)):
            x,y,w,h=rectArr[ind]
            deltaX=int((80-w)/2)
            deltaY=int((80-h)/2)
            if(y-deltaY<0 or x-deltaX<0 or x-deltaX+80>len(baseframe[0]) or y-deltaY+80>len(baseframe)):
                continue
            frame=baseframe.copy()            
            cv2.rectangle(frame,(x-deltaX,y-deltaY),(x-deltaX+80,y-deltaY+80),(0,0,255),2)
            roi=fgmask[y-deltaY:y-deltaY+80,x-deltaX:x-deltaX+80]
            cv2.imshow('asdasdasff',frame)
            key = cv2.waitKey(0)
            classnum=-1
            if(key==ord('q')):
                classnum=0
            if(key==ord('w')):
                classnum=1
            if(key==ord('e')):
                classnum=2
            if(key==ord('r')):
                classnum=3
            if(key==ord('t')):
                dataset_tune()                
            if(classnum>=0):
                blank_image = np.zeros((200,400,3), np.uint8)
                saveMatInDatasetFile(roi,classnum)
                cv2.putText(blank_image,"saved to the class "+
                            getClassName(classnum)+"; count:"+str(len(x_values[0]+x_values[1]+x_values[2]))
                            ,(25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (33, 255, 237), 1)
                cv2.imshow('status window',blank_image)
            if(key==27):
                break
            if(key==32):
                perFrame=False
                cv2.destroyWindow('status window')
                cv2.destroyWindow('asdasdasff')
                break
            ind-=1
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
    
    #cv2.drawContours(black,unified,-1, (255,255,255), -1)
    #cv2.drawContours(frame,unified,-1, (0,255,0), 2)
   
    
    #cv2.imshow('asdasdff',frame)
    
    if(len(rectArr)>0 and perFrame==True):
        continue
    delay = 1
    k = cv2.waitKey(delay)
    if(k!=-1):
        print (k)
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
            cv2.imwrite('picr{:>05}.jpg'.format(iii), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

#cap.release()
cv2.destroyAllWindows()

27


In [25]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model

def find_if_close(cnt1,cnt2,const):
    M1 = cv2.moments(cnt1)
    M2 = cv2.moments(cnt2)
    
    if(M1["m00"]!=0 and M2["m00"]!=0):
        cX1 = int((M1["m10"] / M1["m00"]))
        cY1 = int((M1["m01"] / M1["m00"]))
        cX2 = int((M2["m10"] / M2["m00"]))
        cY2 = int((M2["m01"] / M2["m00"]))
        
        dist = np.sqrt((cX1-cX2)*(cX1-cX2)+(cY1-cY2)*(cY1-cY2)) 
        if dist < const :
            return True,(cX1,cY1),(cX2,cY2)
    return False,(0,0),(0,0)   

def saveMatInDir(mat,dir,name):
    cv2.imwrite(dir+name, mat)
    
def preproc_img(img):
    temp=img.transpose((2, 0, 1))
    return temp[None,:80,:80,...]

def getRectArr(contours):
    LENGTH = len(contours)
    status = np.zeros((LENGTH,1))
    
    for i,cnt1 in enumerate(contours):
        x = i    
        if i != LENGTH-1:
            for j,cnt2 in enumerate(contours[i+1:]):
                x = x+1
                dist,bbbb,bbbbbb = find_if_close(cnt1,cnt2,50)
                if dist == True:
                    val = min(status[i],status[x])
                    status[x] = status[i] = val
                else:
                    if status[x]==status[i]:
                        status[x] = i+1
    rectArr=[]
    if(len(status>0)):
        unified=[]
        maximum = int(status.max())+1
        for i in range(maximum):
            pos = np.where(status==i)[0]
            if pos.size != 0:
                cont = np.vstack(contours[i] for i in pos)
                x,y,w,h = cv2.boundingRect(cont)
                #cv2.rectangle(frame,(x,y),(x+w,y+h),(150,60,60),2)
                rectArr.append((x,y,w,h))
    return rectArr
def getBallOnImg(rectArr,baseframe,fgmask,model):
    frame=baseframe.copy()
    resArr=[]
    x,y,w,h=(-1,-1,-1,-1)
    for ind,it in enumerate(rectArr):
        x,y,w,h=it
        deltaX=int((80-w)/2)
        deltaY=int((80-h)/2)
        if(y-deltaY<0 or x-deltaX<0 or x+w+deltaX>len(baseframe[0]) or y+h+deltaY>len(baseframe)):
            continue     
        roi=fgmask[y-deltaY:y+h+deltaY,x-deltaX:x+w+deltaX]
        #print(roi)
        #PLEASE FORGIVE ME FOR THIS CODE, I MADE A MISTAKE IN CNN TRAINING, TEMPORARY SOLUTION BELOW:
        #----------------
        cv2.imwrite("test.jpg", roi)
        imgSaved=cv2.imread('test.jpg')
        imgSaved=cv2.resize(imgSaved,(80, 80),interpolation=cv2.INTER_NEAREST)
        #----------------
        answer=model.predict(preproc_img(imgSaved))
        if answer[0][0]<0.5:            
            resArr.append((x,y,w,h))
    return resArr

#cap = cv2.VideoCapture('C:\\Users\\egor\\Documents\\20161208_074126.mp4')
cap = cv2.VideoCapture('C:\\Users\\egor\\Documents\\vid.mp4')

#cap = cv2.VideoCapture("D:\\vidsnew\\vid_last.mp4")
cap.set(cv2.CAP_PROP_POS_MSEC,176000)
#cap.set(cv2.CAP_PROP_POS_MSEC,1610000)
fgbg = cv2.createBackgroundSubtractorMOG2(100,30,False)
perFrame=True
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
iii = 30
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
oldUnified=[]
unified = []
linearr=[]
counter=0
currentTrackArr=[]
trackedLevel=0
model = load_model('C:\\Users\\egor\\Documents\\marking\\test_model.kek')
noBallFrameCount=15
while(1):
    iii+=1
    ret, frame = cap.read()
    #frame = cv2.imread('C:\\Users\\egor\\Documents\\github\\diploma\\ball_try\\R_try2\\'+str(iii).zfill(9)+".png",0)
    
    baseframe=frame.copy()
    ##preproc
    mask = np.zeros((frame.shape[0],frame.shape[1]), dtype=np.uint8)
    roi_corners = np.array([[(0,720),(0,460),(480,221),(480,160),(820,160),(820,215),(1280,460),(1280,720)]], dtype=np.int32)
    channel_count = frame.shape[2]  # i.e. 3 or 4 depending on your img
    ignore_mask_color = 255
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    frame = cv2.bitwise_and(frame,frame,mask = mask)
    fgmask = cv2.bitwise_and(fgmask,fgmask,mask = mask)
    ##
    
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    black = np.zeros_like(fgmask)
    res=np.zeros_like(frame)
    #res = cv2.bitwise_and(frame,frame,mask = fgmask)
   
    #fgmask = cv2.dilate(fgmask, kernel1, iterations=5)

    ret,thresh = cv2.threshold(fgmask,1,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if(len(contours)>250):
        continue
    rectArr=getRectArr(contours)
    if(len(rectArr)>50):
        continue
    
    resArr=getBallOnImg(rectArr,baseframe,fgmask,model)
    #for (x,y,w,h) in resArr:
        #cv2.rectangle(baseframe,(x,y),(x+w,y+h),(150,60,60),2)
    nextCandidates=[]
    if len(resArr)==0:
        noBallFrameCount-=1
        if noBallFrameCount==0:
            trackedLevel=0
            currentTrackArr=[]
            noBallFrameCount=3
    else:
        noBallFrameCount=3
        if len(currentTrackArr)==0:
            for (x,y,w,h) in resArr:
                mX=int(x+w/2)
                mY=int(y+h/2)
                currentTrackArr.append([[mX,mY,0]])
        else:            
            for trackInd,elem in enumerate(currentTrackArr):
                fX,fY,fZ=elem[-1]
                indMin=-1
                distMin=2202
                for ind,(x,y,w,h) in enumerate(resArr):
                    mX=int(x+w/2)
                    mY=int(y+h/2)
                    dist=np.hypot(mX-fX,mY-fY)
                    if(dist<distMin):
                        distMin=dist
                        indMin=ind
                if(distMin!=2202):
                    nextCandidates.append([indMin,distMin,trackInd])
        trackedLevel+=1
        
        #print(currentTrackArr)
    for ind,elem in enumerate(nextCandidates):
        for indCur,elemCur in enumerate(nextCandidates):
            if(ind!=indCur):
                if(elemCur[2]!=-1 and elemCur[2]==elem[2]):
                    if elem[1]<elemCur[1]:
                        nextCandidates[indCur][2]=-1
                    else:
                        nextCandidates[ind][2]=-1
                elif(elemCur[2]!=-1 and elemCur[0]==elem[0]):
                    if elem[1]<elemCur[1]:
                        nextCandidates[indCur][2]=-1
                    else:
                        nextCandidates[ind][2]=-1
    for ind,elem in enumerate(nextCandidates):
        if(elem[2]!=-1):
            x,y,w,h=resArr[elem[0]]
            mX=int(x+w/2)
            mY=int(y+h/2)
            currentTrackArr[elem[2]].append([mX,mY,elem[1]])
    
    if(trackedLevel>2):
        maxInd=0
        for (ind,elem) in enumerate(currentTrackArr):
            if len(elem) > len(currentTrackArr[maxInd]):
                maxInd=ind
        currentTrackArr=[currentTrackArr[maxInd]]
        prevPoint=currentTrackArr[0][0]
        for currentPoint in currentTrackArr[0][1:]:   
            firstCenter=prevPoint[0],prevPoint[1]
            secondCenter=currentPoint[0],currentPoint[1]
            cv2.line(baseframe,firstCenter,secondCenter,(0,0,255),2)
            prevPoint=currentPoint
    #if(len(currentTrackArr)>0 and len(currentTrackArr[0])>5):
    #    print(currentTrackArr)
    #    print(np.median([z for x,y,z in currentTrackArr[0]]))
    #    print(np.mean([z for x,y,z in currentTrackArr[0]]))
    #    print(np.std([z for x,y,z in currentTrackArr[0]]))
    #    print(np.var([z for x,y,z in currentTrackArr[0]]))
    #    mean=np.mean([z for x,y,z in currentTrackArr[0]])
    #    std=np.std([z for x,y,z in currentTrackArr[0]])
    #    lenOld=len(currentTrackArr)
    #    currentTrackArr[0]=[item for item in currentTrackArr[0] if (item[2]<mean+std*3)]
    #    trackedLevel=len(currentTrackArr[0])
    #    nonCalc=[(ind,elem) for (ind,elem) in enumerate(currentTrackArr[0]) if elem[2]==0]
    #    print(nonCalc)
    #    for ind,elem in nonCalc:
    #        if(ind<len(currentTrackArr[0])-1):
    #            currentTrackArr[0][ind][2]=np.hypot(currentTrackArr[0][ind+1][0]-currentTrackArr[0][ind][0],
    #                                            currentTrackArr[0][ind+1][1]-currentTrackArr[0][ind][1])
            
                
            
    #cv2.drawContours(black,unified,-1, (255,255,255), -1)
    #cv2.drawContours(frame,unified,-1, (0,255,0), 2)
   
    cv2.imshow('asdasdf',fgmask)
    cv2.imshow('asdasdff',frame)
    cv2.imshow('asdasdffaa',baseframe)
    
    delay = 1
    k = cv2.waitKey(delay)
    
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(iii), res)
            cv2.imwrite('picr{:>05}.jpg'.format(iii), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

cap.release()
cv2.destroyAllWindows()

In [2]:
a=[0]
print(a[-1])

0
